In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [5]:
from datasets import load_dataset
dataset = load_dataset("roberthsu2003/for_llama3_Instruct",split='train')
dataset

Dataset({
    features: ['conversations'],
    num_rows: 52002
})

In [7]:
from pprint import pprint
pprint(dataset[0]['conversations'],compact=True)

[{'content': '給出三個保持健康的提示。', 'role': 'system'}, {'content': '', 'role': 'user'},
 {'content': '1. 飲食均衡，確保攝取足夠的水果和蔬菜。\n2. 定期運動，保持身體活躍和強壯。\n3. 睡眠充足，保持一致的睡眠時間表。',
  'role': 'assistant'}]


### 建立tokenizer和model


# RoPE (Rotary Position Embedding) 擴充說明

RoPE (Rotary Position Embedding) 是一種位置編碼方法，主要用於 Transformer 模型中，讓模型能夠理解序列中token的相對位置關係。以下是詳細說明：

### 基本概念
- RoPE 通過將 token 的向量表示進行旋轉變換來編碼位置信息
- 相比傳統的位置編碼，RoPE 具有更好的外推性能

### 主要特點
1. **相對位置感知**
   - RoPE 能夠讓模型更好地理解 tokens 之間的相對距離關係
   - 不受序列絕對位置的限制

2. **外推能力**
   - 模型在訓練時使用較短序列長度（如2048）
   - 推理時可以處理更長的序列（如4096或更長）

3. **計算效率**
   - 實現簡單且計算開銷較小
   - 不需要額外的記憶體來存儲位置編碼

### 擴充原理


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 #任意選擇！我們內部自動支援 RoPE 擴充！ 

In [ ]:
# 假設原本訓練序列長度為 2048
max_seq_length = 2048

# 可以透過 RoPE 擴充到更長序列
extended_seq_length = 4096  # 擴充後的長度



### 使用場景
- 長文本生成
- 文檔摘要
- 代碼補全
- 其他需要處理長序列的任務

RoPE 擴充使得模型能夠處理比訓練時更長的序列，這對於需要處理長文本的應用場景特別有用。

In [8]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 #任意選擇！我們內部自動支援 RoPE 擴充！
dtype = None #自動偵測,Float16 適用於 Tesla T4、V100，Bfloat16 適用於 Ampere+
load_in_4bit = True #使用4bit量化來減少記憶體使用。可以是false

# 我們支援 4 位元預量化模型，實現 4 倍更快的下載速度 + 無 OOM。
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

AssertionError: Torch not compiled with CUDA enabled